In [1]:
from sklearn.model_selection import KFold

taxi = pd.read_csv("edited_taxi.csv")

In [2]:
def storage(result, remark) :
    """
    회귀분석 결과 모음 
    
    전역변수로 아래 데이터 프레임 선언
    result_sets = pd.DataFrame(columns = ["R square", "AIC", "BIC", "P_fstatics", "P_omnibus", "P_jb", "Cond_no", "changed"])

    Variables_name.fit() 상태로 삽입
    ex)result = model.fit() 이면
    result를 삽입    
    
    remark = 이전 데이터 셋에서 무엇을 변화시켰는지 적을 것.
    """    
    from statsmodels.stats.stattools import durbin_watson
    result.summary()
    put = {
        "R-square" :round(result.rsquared, 3),
        "AIC" : round(result.aic, 3),
        "BIC" : round(result.bic, 3),
        "Pb(Fstatics)" : round(result.f_pvalue, 3),
        "Pb(omnibus)" : round(result.diagn['omnipv'], 3),
        "Pb(jb)" : round(result.diagn['jbpv'], 3),
        "Cond.No." : round(result.diagn['condno'], 3), 
        "Dub-Wat": round(durbin_watson(result.wresid), 3),
        "Remarks" : remark,
    }
    
    result_sets.loc[len(result_sets)] = put

In [3]:
cv = KFold(n_splits=3, shuffle=True, random_state=0)
predict_set = pd.DataFrame()
result_sets = pd.DataFrame(
    columns = ["R-square", "AIC", "BIC", "Cond.No.","Remarks",
                       "Pb(Fstatics)", "Pb(omnibus)", "Pb(jb)", "Dub-Wat"])

In [4]:
for i, (train_index, test_index) in enumerate(cv.split(taxi)):
    model =  sm.OLS.from_formula(
        "np.log(trip_duration) ~ scale(np.log(dist+1))", data=taxi.iloc[train_index])
    result = model.fit()
    storage(result, "test")
    predict_set=pd.concat([predict_set, 
                           result.predict(taxi.iloc[test_index]).reset_index(drop=True)], 
                                              axis=1,ignore_index=True)

In [5]:
result_sets

,R-square,AIC,BIC,Cond.No.,Remarks,Pb(Fstatics),Pb(omnibus),Pb(jb),Dub-Wat
0,0.560,1529894.609,1529918.184,1.0,test,0.0,0.0,0.0,1.998
1,0.560,1524450.433,1524474.008,1.0,test,0.0,0.0,0.0,1.998
2,0.561,1522045.237,1522068.812,1.0,test,0.0,0.0,0.0,2.002


In [6]:
print(predict_set.shape)
predict_set.head()

(486215, 3)


,0,1,2
0,7.175931,6.141959,6.135458
1,7.085067,6.252414,6.013979
2,6.765601,5.975866,6.758605
3,6.269637,6.073878,6.686951
4,5.924230,6.067331,6.223149
